<a href="https://colab.research.google.com/github/akhil-gautam/Calculator/blob/master/mental__health__bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 38.6 MB/s eta 0:00:00


In [12]:
from datasets import load_dataset
import torch

# --- ⚠️ IMPORTANT: REPLACE THIS WITH YOUR DATASET ID ---
DATASET_ID = "YvvonM/mental_health_data"

# Load the dataset
try:
    dataset = load_dataset(DATASET_ID, split="train")
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("\n--- PLEASE MAKE SURE YOU HAVE REPLACED 'your-username/your-dataset-name' ---")
    # As a fallback for demonstration, we'll create a dummy dataset.
    from datasets import Dataset
    dummy_data = {
        "Instruction": ["You are a compassionate and highly skilled mental health counselor.", "You are a compassionate and highly skilled mental health counselor."],
        "Input": ["I've been feeling so sad and overwhelmed lately.", "I can't seem to focus on my work anymore."],
        "Response": ["I'm here to listen and support you. It's brave of you to share.", "That sounds really stressful. Let's talk about what might be causing this."]
    }
    dataset = Dataset.from_dict(dummy_data)
    print("\nLoaded a dummy dataset for demonstration purposes.")


# Define the function to create the training prompt
def create_prompt(example):
    # This function now returns a list of formatted strings
    prompts = []
    for i in range(len(example['Instruction'])):
        prompt = f"""### Instruction:
{example['Instruction'][i]}

### Input:
{example['Input'][i]}

### Response:
{example['Response'][i]}"""
        prompts.append(prompt)
    return prompts

# We no longer apply the map function here.
# The 'dataset' object is now ready for the trainer.

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Model ID
model_id = "Qwen/Qwen2-0.5B-Instruct"

# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare the model for k-bit training and apply LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [9]:
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./qwen2-mental-health-finetuned",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    logging_steps=25,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    max_length=1024,
    return_tensors="pt"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    formatting_func=create_prompt,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,  # <-- controls sequence length
)


TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'